# Site-specific Authentication and Federated Job-level Authorization

Site-specific authentication and authorization allows users to inject their own authentication and authorization methods into the NVFlare system. This includes the FL server / clients registration, authentication, and the job deployment and run authorization.

NVFlare provides a general purpose event based pluggable authentication and authorization framework to allow for expanding functionality such as:

* exposing the app through a WAF (Web Application Firewall) or any other network element enforcing Mutual Transport Layer Security(mTLS)

* using a confidential certification authority to ensure the identity of each participating site and to ensure that they meet the computing requirements for confidential computing

* defining additional roles to manage who can submit which kind of jobs to execute within NVFlare, identify who submits jobs and which dataset can be accessed

Users can write their own FLComponents, listening to the NVFlare system events at different points of their workflow, then easily plug in their authentication and authorization logic as needed.

### Assumptions and Risks
By enabling the customized site-specific authentication and authorization, NVFlare will make several security related data available to the external FL components, e.g. IDENTITY_NAME, PUBLIC_KEY, CERTIFICATE, etc. In order to protect them from being compromised, that data needs to be made read-only.

Because of the external pluginable authentication and authorization processes, the results of the processes could potentially cause the jobs to not be able to be deployed or run. When configuring and using these functions, the users need to be aware of the impact and know where to plug in the authentication and authorization check.

### Event based pluginable authentication and authorization
The NVFlare event based solution supports site-specific authentication and federated job-level authorization. Users can provide and implement any sort of additional security checks by building and plugging in FLcomponents which listen to the appropriate events and provide custom authentication and authorization functions.


To use the site-specific security functions, write a custom Security implementation in the local/custom/security_handler.py, then configure it as a component in the site resources.json.

```
from typing import Tuple

from nvflare.apis.event_type import EventType
from nvflare.apis.fl_component import FLComponent
from nvflare.apis.fl_constant import FLContextKey
from nvflare.apis.fl_context import FLContext
from nvflare.apis.job_def import JobMetaKey


class CustomSecurityHandler(FLComponent):

    def handle_event(self, event_type: str, fl_ctx: FLContext):
        if event_type == EventType.AUTHORIZE_COMMAND_CHECK:
            result, reason = self.authorize(fl_ctx=fl_ctx)
            if not result:
                fl_ctx.set_prop(FLContextKey.AUTHORIZATION_RESULT, False, sticky=False)
                fl_ctx.set_prop(FLContextKey.AUTHORIZATION_REASON, reason, sticky=False)

    def authorize(self, fl_ctx: FLContext) -> Tuple[bool, str]:
        command = fl_ctx.get_prop(FLContextKey.COMMAND_NAME)
        if command in ["check_resources"]:
            security_items = fl_ctx.get_prop(FLContextKey.SECURITY_ITEMS)
            job_meta = security_items.get(FLContextKey.JOB_META)
            if job_meta.get(JobMetaKey.JOB_NAME) == "FL Demo Job1":
                return False, f"Not authorized to execute: {command}"
            else:
                return True, ""
        else:
            return True, ""

```

# Site-specific Authentication & Authorization

Lets look these mechanism via examples

## Server Side Customized Authenticaton

In this example, we will see how do we designed a custom plugin with additional authentication check, 
As result for two sites in POC, site-2 is NOT able to start and register to the server. It's blocked by the ServerCustomSecurityHandler logic during the client registration.

### Define a server side security handler

Notice the we the customized the handler raise NotAuthenticated("site_2 not allowed to register")

In [ ]:
!cat examples/custom_server_side_authentication/security/server/custom/security_handler.py

To register this plugin handler, we need to add this component to the server site's local configuration

by adding it to the components array 

```
    components: [
    ...
            {
                "id": "security_handler",
                "path": "security_handler.ServerCustomSecurityHandler"
            }
    ]  
```  

In this example, we have made a edit resources.json file and will copy both "custom" folder and "resources.json" to the local directory



In [ ]:
! echo y | nvflare poc prepare

!cp -r examples/custom_server_side_authentication/security/server/* /tmp/nvflare/poc/example_project/prod_00/server/local/.



Now, go to a terminal and try to start FL system with 

```

nvflare poc start -ex admin@nvidia.com
```

See what happens

You should see something like this: 


```
2025-02-02 16:35:40,059 - Communicator - INFO - Trying to register with server ...
2025-02-02 16:35:40,060 - ServerCustomSecurityHandler - ERROR - [identity=server, run=?, peer=site-2, peer_run=?] - Exception when handling event "_client_register_received": NotAuthenticated: site-2 not allowed to register


```

In [ ]:
# Clean up

! nvflare poc clean

## Client Side: Customized Job-level Auhtorization

Next, let's take look authorization on the client side

**Setup**

* `server`: NVFlare server
* `site-1`: Site-1 has a CustomSecurityHandler set up which does not allow the job "secret-job" to run. All other jobs will be able to deploy and run on site-1.
* `site-2`: Site-2 allows any job to be deployed and run.

**Expectation**
* "secret-job" will be deployed and run on site-2 but not on site-1


What we will do: 

* install dependencies
* download data
* generate two job configs. We can use fl_jobs.py


In [ ]:
# install dependencies

! pip install -r examples/custom_client_side_job_level_authorization/code/requirements.txt

# download data

! python examples/custom_client_side_job_level_authorization/code/data/download.py

In [ ]:
%cd examples/custom_client_side_job_level_authorization/code

! python fl_jobs.py

# change back
%cd - 


In [ ]:
!cat   /tmp/nvflare/jobs/workdir/fedavg/app_site-1/config/config_fed_client.json 

Next we 
* create a POC workspace and
* then install the customized securitry handler to site-1,
* edit site-1/local/resources.json to add security handler component 
> note: 
  to simplify, we just copy the pre-edit resources.json to that location




In [ ]:
# prepare poc
! echo y | nvflare poc prepare


In [45]:
# cp security handler and component config
!cp -r examples/custom_client_side_job_level_authorization/security/site-1/* /tmp/nvflare/poc/example_project/prod_00/site-1/local/.


Now we are ready to run the job. 


* start poc

    Use a terminal ( note notebook cell) start the poc with the following command

    ```
    nvflare poc start -ex admin@nvidia.com 

    ```

    this bring up the FL system 



In [ ]:
#  Submit jobs
#  Assuming at this point FL system is already running via poc start command

!  nvflare job submit -j /tmp/nvflare/jobs/workdir/fedavg

# The job should finish as expected

In [ ]:
#  Submit jobs
#  Assuming at this point FL system is already running via poc start command

!  nvflare job submit -j /tmp/nvflare/jobs/workdir/secret-job

you should get something like

```
2025-02-02 20:31:03,494 - site_security - ERROR - Authorization failed. Reason: Not authorized to execute: check_resources
2025-02-02 20:31:03,496 - ServerEngine - ERROR - Client reply error: Not authorized to execute: check_resources

```

* Cleanup 

In [ ]:

! nvflare poc stop

In [ ]:
! nvflare poc clean

## Client Side: Integration with authentication system

#### Overview

In Federated Computing systems, many participating institutions already have their own in-house authentication systems. Instead of introducing a new authentication mechanism, we need to integrate with them. Note that this integration can be site-specific; in other words, each site may be different. Site-1 can be using OLAP, site-2 can use OAuth, and a third site can use something else.

In this example, we demonstrate NVIDIA FLARE's event-based plugin-component that can be used to integrate any type of authentication/authorization mechanism, using open-source KeyCloak integration as an example.

### Setup KeyCloak
<!-- 
Download and install the KeyCloak following the instruction guide from https://www.keycloak.org/getting-started/getting-started-zip. Key steps for the installation:

<!-- markdown-link-check-disable -->

* Start the KeyCLoak by running "bin/kc.sh start-dev"
* Set up the realm called "myrealm"
* Setup the user "myuser@example.com"
* Add the client "myclient"
* The KeyCLoak will be running at http://localhost:8080/

 -->

> pre-requisits
* need to install JDK first
* skip this step if you already have JDK installed 

#### JDK Installatioon 

``` 
sudo apt install -y openjdk-21-jdk

java -version
javac -version

```



In [10]:
VERSION = "26.1.0"
INSTALL_DIR = "/tmp/keycloak"
DOWNLOAD_URL = f"https://github.com/keycloak/keycloak/releases/download/{VERSION}/keycloak-{VERSION}.zip"
ZIP_PATH = f"/tmp/keycloak-{VERSION}.zip"

In [ ]:
# Download Keycloak

!curl -Lo {ZIP_PATH} {DOWNLOAD_URL}
! ls -al /tmp/keycloak-*


In [ ]:
! echo A | unzip {ZIP_PATH} -d /tmp

In [13]:
! mv /tmp/keycloak-{VERSION} {INSTALL_DIR}

In [ ]:
# setup admin user

!cd {INSTALL_DIR} && ./bin/kc.sh create-admin --user admin --password admin123
